In [1]:
from sklearn.datasets import load_boston
!pip install optuna

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
import xgboost as xgb
# sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

import sklearn.metrics

from mlxtend.classifier import StackingCVClassifier
!pip install vecstack
from vecstack import stacking

from itertools import combinations 


import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

In [3]:
# X, y = load_boston(return_X_y=True)
# X.shape

In [4]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv'
data = pd.read_csv(url,header=1)
data = data.loc[:,['Date','Open',	'High',	'Low', 'Close',	'Volume ETH']]
data = data.rename(columns={"Volume ETH": "Volume"})
data.index = data['Date']
data = data.drop('Date', axis = 1)
data = data.iloc[::-1]
data.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2020-07-08 10-AM,241.09,244.27,241.09,243.60,59031.54
2020-07-08 11-AM,243.60,244.60,242.48,244.07,51011.61
2020-07-08 12-PM,244.07,244.70,241.28,242.06,59565.50
2020-07-08 01-PM,242.06,246.90,242.00,246.32,91679.67
2020-07-08 02-PM,246.32,247.63,245.64,246.14,48127.75


In [5]:
def get_indicators(data):
    # Previous Candle
    for i in np.arange(1,21,1):
      data[f'Close_lag{i}'] = data['Close'].shift(i)
      data[f'Volume_lag{i}'] = data['Volume'].shift(i)


    data['target'] = data['Close'] #.shift(-1)
    data = data.drop(['Open',	'High',	'Low', 'Close',	'Volume'], axis=1)

    return data

data = get_indicators(data)
data = data[data['Close_lag20'].notna()]
data.head(3)

,Close_lag1,Volume_lag1,Close_lag2,Volume_lag2,Close_lag3,Volume_lag3,Close_lag4,Volume_lag4,Close_lag5,Volume_lag5,Close_lag6,Volume_lag6,Close_lag7,Volume_lag7,Close_lag8,Volume_lag8,Close_lag9,Volume_lag9,Close_lag10,Volume_lag10,Close_lag11,Volume_lag11,Close_lag12,Volume_lag12,Close_lag13,Volume_lag13,Close_lag14,Volume_lag14,Close_lag15,Volume_lag15,Close_lag16,Volume_lag16,Close_lag17,Volume_lag17,Close_lag18,Volume_lag18,Close_lag19,Volume_lag19,Close_lag20,Volume_lag20,target
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-08-18 12-AM,303.02,119.21,306.74,368.37,307.44,330.55,304.79,270.32,309.41,215.64,309.10,203.65,308.33,191.95,308.00,596.80,307.06,459.42,306.50,695.46,309.40,310.58,308.62,401.78,308.95,271.06,309.30,284.39,310.00,464.18,308.62,150.75,307.96,753.19,302.68,302.51,303.10,377.67,300.79,122.52,301.19
2017-08-18 01-AM,301.19,496.43,303.02,119.21,306.74,368.37,307.44,330.55,304.79,270.32,309.41,215.64,309.10,203.65,308.33,191.95,308.00,596.80,307.06,459.42,306.50,695.46,309.40,310.58,308.62,401.78,308.95,271.06,309.30,284.39,310.00,464.18,308.62,150.75,307.96,753.19,302.68,302.51,303.10,377.67,298.73
2017-08-18 02-AM,298.73,320.05,301.19,496.43,303.02,119.21,306.74,368.37,307.44,330.55,304.79,270.32,309.41,215.64,309.10,203.65,308.33,191.95,308.00,596.80,307.06,459.42,306.50,695.46,309.40,310.58,308.62,401.78,308.95,271.06,309.30,284.39,310.00,464.18,308.62,150.75,307.96,753.19,302.68,302.51,300.52


In [23]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1:].values.ravel()

In [24]:
print(X.shape)
print(y.shape)

(25335, 40)
(25335,)


Creating Features 

In [8]:
X_TSNE = TSNE(n_components=2).fit_transform(X)
X_DBSCAN = DBSCAN(eps=3, min_samples=2).fit(X)
X_PCA = PCA(n_components=2).fit_transform(X)


# add up to 1024 depending on how large your data is PICK 1 
# X_KNN2, indices = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X).kneighbors(X)
# X_KNN4, indices = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X).kneighbors(X)
# X_KNN8, indices = NearestNeighbors(n_neighbors=8, algorithm='ball_tree').fit(X).kneighbors(X)
# X_KNN16, indices = NearestNeighbors(n_neighbors=16, algorithm='ball_tree').fit(X).kneighbors(X)
# X_KNN32, indices = NearestNeighbors(n_neighbors=32, algorithm='ball_tree').fit(X).kneighbors(X)
X_KNN64, indices = NearestNeighbors(n_neighbors=64, algorithm='ball_tree').fit(X).kneighbors(X)
# X_KNN128, indices = NearestNeighbors(n_neighbors=128, algorithm='ball_tree').fit(X).kneighbors(X)

In [25]:
# USING PANDAS

# KNN : YOU ONLY NEED TO PICK 1 OF THE KNNs ABOVE.

# X['KNN2'] = X_KNN2[:,1::]

# for i in np.arange(1,4,1):
#   X[f'KNN4_{i}'] = X_KNN4[:,i]

for i in np.arange(1,64,1):
  X[f'KNN64_{i}'] = X_KNN64[:,i]

# replace na values

# for i in np.arange(0,2,1):
#   X[f'TSNE_{i}'] = X_TSNE[:,i]

# X['DBSCAN'] = X_DBSCAN.labels_

# for i in np.arange(0,2,1):
#   X[f'PCA_{i}'] = X_PCA[:,i]


X.shape

(25335, 103)

In [10]:
X.tail()

,Close_lag1,Volume_lag1,Close_lag2,Volume_lag2,Close_lag3,Volume_lag3,Close_lag4,Volume_lag4,Close_lag5,Volume_lag5,Close_lag6,Volume_lag6,Close_lag7,Volume_lag7,Close_lag8,Volume_lag8,Close_lag9,Volume_lag9,Close_lag10,Volume_lag10,Close_lag11,Volume_lag11,Close_lag12,Volume_lag12,Close_lag13,Volume_lag13,Close_lag14,Volume_lag14,Close_lag15,Volume_lag15,Close_lag16,Volume_lag16,Close_lag17,Volume_lag17,Close_lag18,Volume_lag18,Close_lag19,Volume_lag19,Close_lag20,Volume_lag20,...,KNN64_29,KNN64_30,KNN64_31,KNN64_32,KNN64_33,KNN64_34,KNN64_35,KNN64_36,KNN64_37,KNN64_38,KNN64_39,KNN64_40,KNN64_41,KNN64_42,KNN64_43,KNN64_44,KNN64_45,KNN64_46,KNN64_47,KNN64_48,KNN64_49,KNN64_50,KNN64_51,KNN64_52,KNN64_53,KNN64_54,KNN64_55,KNN64_56,KNN64_57,KNN64_58,KNN64_59,KNN64_60,KNN64_61,KNN64_62,KNN64_63,TSNE_0,TSNE_1,DBSCAN,PCA_0,PCA_1
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-08 10-AM,241.09,26613.96,240.76,25039.21,240.76,35187.07,239.99,16661.54,239.19,25651.65,239.20,58665.56,241.90,36051.36,241.47,50633.30,240.12,37584.31,238.46,22333.50,239.39,18448.02,238.93,15342.06,238.13,13973.63,238.49,24130.58,237.98,11389.74,238.90,13262.34,239.87,19892.30,238.42,27691.97,238.82,25898.23,237.20,56615.60,...,55390.743042,55421.722085,55479.903551,55513.853873,55734.536165,55872.599281,55983.645458,56240.174934,56311.348089,56485.300098,56604.847288,56654.583764,56656.334276,56763.344919,56805.438802,56958.478325,57139.913168,57197.046647,57312.512940,57422.846385,57565.204659,57638.467261,57945.455498,57960.927118,58337.766226,58384.331120,58522.915196,58618.257593,58656.918935,58663.302668,58742.013754,58860.871034,58906.373102,59035.430038,59049.821905,13.549782,40.279243,-1,60110.995059,21411.725651
2020-07-08 11-AM,243.60,59031.54,241.09,26613.96,240.76,25039.21,240.76,35187.07,239.99,16661.54,239.19,25651.65,239.20,58665.56,241.90,36051.36,241.47,50633.30,240.12,37584.31,238.46,22333.50,239.39,18448.02,238.93,15342.06,238.13,13973.63,238.49,24130.58,237.98,11389.74,238.90,13262.34,239.87,19892.30,238.42,27691.97,238.82,25898.23,...,58009.078118,58037.373946,58119.123856,58123.374270,58138.090448,58189.772745,58306.980872,58549.852081,58599.249136,58610.632610,58622.505921,58729.322460,58795.191343,58856.073826,58924.361940,58940.573601,59032.516957,59075.453740,59289.677772,59335.554637,59352.656477,59419.923832,59446.767621,59484.873354,59554.921668,59577.566802,59604.203088,59628.875442,59635.057100,59789.050766,59913.002132,60152.035526,60165.066061,60351.980219,60385.844103,10.269239,40.784504,-1,60711.434913,33099.896533
2020-07-08 12-PM,244.07,51011.61,243.60,59031.54,241.09,26613.96,240.76,25039.21,240.76,35187.07,239.99,16661.54,239.19,25651.65,239.20,58665.56,241.90,36051.36,241.47,50633.30,240.12,37584.31,238.46,22333.50,239.39,18448.02,238.93,15342.06,238.13,13973.63,238.49,24130.58,237.98,11389.74,238.90,13262.34,239.87,19892.30,238.42,27691.97,...,60549.563996,60590.267376,60751.791503,60771.024477,60805.016405,60816.652279,60944.533720,61090.866645,61537.469177,61645.380133,61678.974779,61778.746759,61817.816792,61917.143766,62103.648386,62113.996735,62156.723466,62262.900297,62491.445136,62575.353624,62634.837868,62686.931148,62778.539993,62832.844501,63155.086679,63176.888062,63233.963982,63265.198782,63301.307206,63304.742688,63329.095844,63338.869086,63356.429343,63497.509547,63557.418434,10.729051,41.372658,-1,66334.221015,36936.069516
2020-07-08 01-PM,242.06,59565.50,244.07,51011.61,243.60,59031.54,241.09,26613.96,240.76,25039.21,240.76,35187.07,239.99,16661.54,239.19,25651.65,239.20,58665.56,241.90,36051.36,241.47,50633.30,240.12,37584.31,238.46,22333.50,239.39,18448.02,238.93,15342.06,238.13,13973.63,238.49,24130.58,237.98,11389.74,238.90,13262.34,239.87,19892.30,...,65181.808810,65293.007473,65376.760393,65383.924785,65841.383049,66137.542100,66763.917034,66881.250158,66967.780509,66977.901464,67059.200757,67129.093101,67325.970436,67336.473401,67365.726590,67453.94

In [11]:
# USING NUMPY ARRAYS
# X = np.concatenate((X, X_TSNE), axis=1)
# X = np.c_[ X, X_DBSCAN.labels_]
# X = np.concatenate((X, X_PCA), axis=1)
# X = np.concatenate((X, X_KNN2[:,1::]), axis=1)
# X = np.concatenate((X, X_KNN4[:,1::]), axis=1)
# X = np.concatenate((X, X_KNN8[:,1::]), axis=1)
# X = np.concatenate((X, X_KNN16[:,1::]), axis=1)
# X = np.concatenate((X, X_KNN32[:,1::]), axis=1)
# X = np.concatenate((X, X_KNN64[:,1::]), axis=1)
# X = np.concatenate((X, X_KNN128[:,1::]), axis=1)
# X.shape






In [12]:
def scaler_fuc(scaler):
  train_x, valid_x, train_y, valid_y = train_test_split(X, y,
                                              test_size=0.25, random_state = 123)
  if (scaler == 'minmax'):
    scaler = MinMaxScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'stand'):
    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'log'):
    transformer = FunctionTransformer(np.log1p, validate=True)
    train_x = transformer.transform(train_x)
    valid_x = transformer.transform(valid_x)

  # turning these train/tests into lgb/xgb datasets
  dtrain_gbm = lgb.Dataset(train_x, label=train_y)
  dvalid_gbm = lgb.Dataset(valid_x, label=valid_y)

  dtrain_xbg = xgb.DMatrix(train_x, label=train_y)
  dvalid_xbg = xgb.DMatrix(valid_x, label=valid_y)
  return train_x, valid_x, train_y, valid_y,  dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg

In [13]:
models = ['lreg', 'las', 'rid','ada', 'et', 'rf', 'xgb', 'gbm'] # when you want to try all of them/ iteration: 1 
# models = ['ada', 'et', 'rf', 'xgb', 'gbm'] # selecting the 5 best models from iteration 1 (by observing the log) / iteration: 2
# models = ['las', 'rid', 'et', 'xgb'] 
comb = combinations(models, 3) 
comb = list(comb)

In [14]:

class Objective:

    def __init__(self):
        self.best_gbm = None
        self._gbm = None
        self.best_xgb = None
        self._xgb = None
        self.predictions = None
        self.fpredictions = None

    def __call__(self, trial):

        i = trial.suggest_int("combos", 0, (len(comb)-1))
 
        gbm_preds = 0
        gbm_rmse = float('inf')
        xgb_preds = 0
        xgb_rmse = float('inf')
        rf_preds = 0
        rf_rmse = float('inf')
        et_preds = 0
        et_rmse = float('inf')
        ada_preds = 0
        ada_rmse = float('inf')
        lreg_preds = 0
        lreg_rmse = float('inf')
        las_preds = 0
        las_rmse = float('inf')
        rid_preds = 0
        rid_rmse = float('inf')

        ###############################################################################
        #                                 . Scaling                                   #
        ###############################################################################
        # If you want 1 scaled data for everything use this and comment out, other wise leave this commented
        # scaler = trial.suggest_categorical("Scaler", ['minmax','stand','log'])
        # train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(scaler)
        ###############################################################################
        #                            . Linear Regression                              #
        ###############################################################################
        if any(x == 'lreg' for x in comb[i]):
          lr_scaler = trial.suggest_categorical("lr_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(lr_scaler)
          lreg = LinearRegression().fit(train_x, train_y)
          lreg_preds = lreg.predict(valid_x)
          lreg_rmse = sklearn.metrics.mean_squared_error(valid_y, lreg_preds, squared = False)
        ###############################################################################
        #                                 . Lasso                                     #
        ###############################################################################
        if any(x == 'las' for x in comb[i]):
          las_scaler = trial.suggest_categorical("las_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(las_scaler)
          alphal = trial.suggest_loguniform("alphal", 1e-8, 1.0)
          las = linear_model.Lasso(alpha=alphal).fit(train_x, train_y)
          las_preds = las.predict(valid_x)
          las_rmse = sklearn.metrics.mean_squared_error(valid_y, las_preds, squared = False)
        ###############################################################################
        #                                 . Ridge                                     #
        ###############################################################################
        if any(x == 'rid' for x in comb[i]):
          rid_scaler = trial.suggest_categorical("rid_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rid_scaler)
          alphar = trial.suggest_loguniform("alphar", 1e-8, 1.0)
          rid = Ridge(alpha=alphar).fit(train_x, train_y)
          rid_preds = rid.predict(valid_x)
          rid_rmse = sklearn.metrics.mean_squared_error(valid_y, rid_preds, squared = False)
        ###############################################################################
        #                               . Ada Boost                                   #
        ###############################################################################
        if any(x == 'ada' for x in comb[i]):
          ada_scaler = trial.suggest_categorical("ada_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(ada_scaler)
          ada_ne = trial.suggest_int("ada_ne", 1, 500) #500
          ada_lr = trial.suggest_loguniform("ada_lr", 0.05, 1)
          ada = AdaBoostRegressor(n_estimators = ada_ne, learning_rate = ada_lr,
                                     random_state=0).fit(train_x, train_y)
          ada_preds = ada.predict(valid_x)
          ada_rmse = sklearn.metrics.mean_squared_error(valid_y, ada_preds, squared = False)
        ###############################################################################
        #                              . Extra Trees                                  #
        ###############################################################################
        if any(x == 'et' for x in comb[i]):
          et_scaler = trial.suggest_categorical("et_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(et_scaler)
          et_md = trial.suggest_int("et_max_depth", 1, 100)
          et_ne = trial.suggest_int("et_ne", 1, 500) #1000
          et = ExtraTreesRegressor(max_depth=et_md, n_estimators = et_ne,
                                     random_state=0).fit(train_x, train_y)
          et_preds = et.predict(valid_x)
          et_rmse = sklearn.metrics.mean_squared_error(valid_y, et_preds, squared = False)
        ###############################################################################
        #                             . Random Forest                                 #
        ###############################################################################
        if any(x == 'rf' for x in comb[i]):
          rf_scaler = trial.suggest_categorical("rf_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rf_scaler)
          rf_md = trial.suggest_int("rf_max_depth", 1, 100)
          rf_ne = trial.suggest_int("rf_ne", 1, 500) #1000
          rf = RandomForestRegressor(max_depth=rf_md, n_estimators = rf_ne,
                                     random_state=0).fit(train_x, train_y)
          rf_preds = rf.predict(valid_x)
          rf_rmse = sklearn.metrics.mean_squared_error(valid_y, rf_preds, squared = False)
        ###############################################################################
        #                                 . XGBoost                                   #
        ###############################################################################
        if any(x == 'xgb' for x in comb[i]): 
          xgb_scaler = trial.suggest_categorical("xgb_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(xgb_scaler)
          xgb_param = {
              "silent": 1,
              "objective": "reg:squarederror",
              "eval_metric": "rmse",
              "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
              "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
              "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
          }
          if xgb_param["booster"] == "gbtree" or xgb_param["booster"] == "dart":
              xgb_param["max_depth"] = trial.suggest_int("max_depth", 1, 100)
              xgb_param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
              xgb_param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
              xgb_param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
          if xgb_param["booster"] == "dart":
              xgb_param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
              xgb_param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
              xgb_param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
              xgb_param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)
          # Add a callback for pruning.
          xgb_pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse" )
          xgb_ = xgb.train(xgb_param, dtrain_xbg, evals=[(dvalid_xbg, "validation")], verbose_eval=False, callbacks=[xgb_pruning_callback])
          xgb_preds = xgb_.predict(dvalid_xbg)
          xgb_rmse = sklearn.metrics.mean_squared_error(valid_y, xgb_preds, squared = False)
          self._xgb = xgb_
        ###############################################################################
        #                          . Light Gradient Boosting                          #
        ###############################################################################
        if any(x == 'gbm' for x in comb[i]):
          gbm_scaler = trial.suggest_categorical("gbm_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(gbm_scaler)
          gbm_param = {
            'objective': 'regression',
            'metric': 'rmse',
              "verbosity": -1,
              "boosting_type": "gbdt",
              "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10), 
              "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10),
              "num_leaves": trial.suggest_int("num_leaves", 2, 256), 
              "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0), 
              "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
              "bagging_freq": trial.suggest_int("bagging_freq", 1, 7), 
              "min_child_samples": trial.suggest_int("min_child_samples", 2, 20), 
          }
          # Add a callback for pruning.
          gbm_pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
          gbm = lgb.train(gbm_param, dtrain_gbm, valid_sets=[dvalid_gbm], verbose_eval=False, callbacks=[gbm_pruning_callback])
          gbm_preds = gbm.predict(valid_x)
          gbm_rmse = sklearn.metrics.mean_squared_error(valid_y, gbm_preds, squared = False)
          self._gbm = gbm
        ###############################################################################
        #                            . Stacking Strategy                              #
        ###############################################################################

        # strat = trial.suggest_int("strat", 0, 1)

        # if (strat == 0): # taking mean
        preds = (gbm_preds + xgb_preds + rf_preds + \
        las_preds + ada_preds + et_preds + \
        lreg_preds + rid_preds) / 3

        # if (strat == 1): # Making an equation
        # a = trial.suggest_uniform("a", 0.1, 0.9)
        # b = trial.suggest_uniform("b", 0.1, 0.9)
        # c = trial.suggest_uniform("c", 0.1, 0.9)
        # d = trial.suggest_uniform("d", 0.1, 0.9)

        # def custom_sort(t):
        #   return t[0]

        # R = [[gbm_rmse,gbm_preds] , [xgb_rmse,xgb_preds], [rf_rmse, rf_preds], \
        #     [las_rmse,las_preds], [ada_rmse,ada_preds],[et_rmse,et_preds],\
        #     [lreg_rmse,lreg_preds], [rid_rmse,rid_preds]]
        # R.sort(key=custom_sort) # low to high
        # m1 = R[0][1]
        # m2 = R[1][1]
        # m3 = R[2][1]
        # # Using list comprehension. This may result in a divide by 0 error when rounding occurs.
        # # A fix would be to use a more precise library
        # preds = [ (a*((x**b) + (y**c))) + (d*z) for x,y,z in zip(m1,m2,m3) ]
        self.predictions = preds
        rmse = sklearn.metrics.mean_squared_error(valid_y, preds, squared = False)
        return rmse

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_gbm = self._gbm
            self.best_xgb = self._xgb
            self.fpredictions = self.predictions

In [26]:
import optuna
objective = Objective()

# Setting SEED 
from optuna.samplers import TPESampler
sampler = TPESampler(seed=10)

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize",
    sampler=sampler
)
study.optimize(objective, n_trials=10, callbacks=[objective.callback]) # change this to 500 + 

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_gbm = objective.best_gbm
best_xgb = objective.best_xgb

[I 2020-07-11 04:41:17,537] Finished trial#0 with value: 126.57800321246911 with parameters: {'combos': 9, 'lr_Scaler': 'stand', 'rid_Scaler': 'minmax', 'alphar': 9.051476667906094e-05, 'xgb_Scaler': 'stand', 'booster': 'gbtree', 'lambda': 9.728728830009641e-05, 'alpha': 6.285982451182996e-07, 'max_depth': 9, 'eta': 1.3320760321578133e-06, 'gamma': 1.2847622527426482e-06, 'grow_policy': 'depthwise'}. Best is trial#0 with value: 126.57800321246911.
[I 2020-07-11 04:44:45,842] Finished trial#1 with value: 9.52028670975306 with parameters: {'combos': 36, 'rid_Scaler': 'minmax', 'alphar': 0.034104735191002086, 'ada_Scaler': 'log', 'ada_ne': 345, 'ada_lr': 0.5704456208852126, 'et_Scaler': 'log', 'et_max_depth': 34, 'et_ne': 201}. Best is trial#1 with value: 9.52028670975306.
[I 2020-07-11 04:46:59,255] Finished trial#2 with value: 24.311391864157027 with parameters: {'combos': 36, 'rid_Scaler': 'log', 'alphar': 0.006599090886310522, 'ada_Scaler': 'log', 'ada_ne': 78, 'ada_lr': 0.07413119546

Best trial:
  Params: 
    combos: 36
    rid_Scaler: minmax
    alphar: 0.034104735191002086
    ada_Scaler: log
    ada_ne: 345
    ada_lr: 0.5704456208852126
    et_Scaler: log
    et_max_depth: 34
    et_ne: 201


In [ ]:
comb[0]

best result so far:

**BINANCE ETH-USD**


```
6.55939309007308
    ('las', 'rid', 'et')
    las_Scaler: stand
    alphal: 0.0004305476461811444
    rid_Scaler: stand
    alphar: 2.951911001774111e-07
    et_Scaler: log
    et_max_depth: 35
    et_ne: 13
```



**BOSTON HOUSE** 

3.067 : ('ada', 'et', 'xgb')

```
  Params: 
    combos: 1
    ada_Scaler: log
    ada_ne: 173
    ada_lr: 0.6716180264894074
    et_Scaler: stand
    et_max_depth: 10
    et_ne: 593
    xgb_Scaler: log
    booster: gbtree
    lambda: 0.056298103046901415
    alpha: 0.0005885345642231615
    max_depth: 3
    eta: 0.5865796691943621
    gamma: 6.697097340043158e-08
    grow_policy: depthwise
```





In [ ]:
objective.fpredictions 